In [1]:
from llama_cpp import Llama, LogitsProcessorList
import numpy as np

In [2]:
!ls /data/ai_club/llms -lah # see llms you can use. Bigger models need more GPUs

total 89G
drwxrwsr-x  2 grall    ai_club 4.0K Apr  2  2024 .
drwxrws--- 39 sysadmin ai_club 4.0K Oct 25 16:15 ..
-rwxrwxr-x  1 grall    grall   6.4G Oct 29  2023 ggml-model-f16.gguf
-rwxrwxr-x  1 grall    grall   4.5G Sep  4  2023 llama-2-7b-chat.Q5_K_M.gguf
-rwxrwxr-x  1 grall    grall   7.2G Feb 19  2024 mistral-7b-instruct-v0.2.Q8_0.gguf
-rwxrwxr-x  1 grall    grall    36G Apr  2  2024 mixtral-8x7b-instruct-v0.1.Q6_K.gguf
-rwxrwxr-x  1 grall    grall    35G Apr  2  2024 yi-34b.Q8_0.gguf


In [2]:
llm = Llama(
    model_path='/data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf',
    n_gpu_layers=-1, # put everything on GPU
    verbose=1,
    n_ctx=1000, # 4000
    logits_all=True
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7: 

---

In [10]:
llm.tokenize(b'test', add_bos=False) # some words are multiple tokens

[1369]

---

In [29]:
resp = llm(
    "this",
    max_tokens=1, # change this!
    temperature=0 # change this!
)
resp

Llama.generate: 1 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     176.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      55.74 ms /     2 tokens


{'id': 'cmpl-6ed95800-93a8-404a-a731-d7177b1369e4',
 'object': 'text_completion',
 'created': 1730403009,
 'model': '/data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf',
 'choices': [{'text': ' is',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 2, 'completion_tokens': 1, 'total_tokens': 3}}

In [30]:
print(
    resp['choices'][0]['text']
)

 is


---

In [40]:
def logits_processor(prev_tok_ids, next_tok_logits):
    # example, zero everything but 'um'
    # next_tok_logits = next_tok_logits*0
    # next_tok_logits[3205] = 100

    # example, adds bias to 'um'
    next_tok_logits[3205] += 1
 
    return next_tok_logits

resp = llm(
    "this is um ",
    max_tokens=10,
    logprobs=5, # Show top 5 most likely tokens
    logits_processor=[logits_processor],
    temperature=0
)

Llama.generate: 4 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     176.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    10 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     422.17 ms /    11 tokens


In [41]:
print(resp['choices'][0]['text']) # llm output
print()
print(resp['choices'][0]['logprobs']['tokens']) # llm output tokens
print()
for lps in resp['choices'][0]['logprobs']['top_logprobs']: # llm output top-N logprobs. This data could lend a good explanation of temperature
    print(lps)

100% a question for the experts here

['1', '0', '0', '%', ' a', ' question', ' for', ' the', ' experts', ' here']

{'1': np.float32(-1.6013106), '2': np.float32(-1.8831013), '3': np.float32(-2.6544409), 'ive': np.float32(-2.7861729), '0': np.float32(-2.8277216)}
{'0': np.float32(-1.4950409), '2': np.float32(-2.0479078), '1': np.float32(-2.1067858), '9': np.float32(-2.6830473), '3': np.float32(-2.7920685)}
{'0': np.float32(-1.2913281), ' minutes': np.float32(-2.54077), '1': np.float32(-2.883245), ' days': np.float32(-3.369475), '.': np.float32(-3.399469)}
{'%': np.float32(-0.4586866), ' percent': np.float32(-2.3217494), '0': np.float32(-2.5390441), '1': np.float32(-4.437709), ' times': np.float32(-4.6682396)}
{' a': np.float32(-2.4757462), ' my': np.float32(-2.5984821), ' true': np.float32(-2.6285858), ' not': np.float32(-3.194232), ' off': np.float32(-3.290863)}
{' question': np.float32(-3.0960052), ' r': np.float32(-3.2916553), ' copy': np.float32(-3.8266113), ' no': np.float32(-3.85